In [ ]:
import numpy as np

In [ ]:
class Dense:

  def __init__(self, input_size, output_size):
    ni = input_size
    no = output_size
    self.W = np.sqrt(2/ni)*np.random.randn(ni,no)
    self.b = np.zeros(no)
    self.x = None
    self.dW = None
    self.db = None

  def forward(self, x):
    self.x = x
    out = np.dot(x, self.W) + self.b
    return out

  def backward(self, dL):
    dx = np.dot(dL, self.W.T)
    self.dW = np.dot(self.x.T, dL)
    self.db = np.sum(dL, axis=0)
    return dx

  def update(self, lr=0.001):
    self.W -= lr*self.dW
    self.b -= lr*self.db

In [ ]:
class ReLu:
  def __init__(self):
    self.mask = None

  def forward(self, x):
    self.mask = (x<=0)
    out = x.copy()
    out[self.mask] = 0
    return out

  def backward(self, dL):
    dL[self.mask] = 0
    return dL

  def update(self, lr = 0.001):
    pass

In [ ]:
class MSE:
  def __init__(self):
    self.y = None
    self.t = None

  def forward(self, y, t):
    if y.ndim != t.ndim:
      t = t.reshape(t.shape[0],1)
    self.t = t
    self.y = y
    mse = (y-t)**2
    return np.average(mse)

  def backward(self):
    batch_size = self.t.shape[0]
    dy = (self.y-self.t) / batch_size
    return dy

In [ ]:
class ANN:
  def __init__(self):

    self.Layers = []

    self.Layers.append(Dense(6,12))
    self.Layers.append(ReLu())
    self.Layers.append(Dense(12,8))
    self.Layers.append(ReLu())
    self.Layers.append(Dense(8,6))
    self.Layers.append(ReLu())
    self.Layers.append(Dense(6,2))

    self.RevLayers = self.Layers.copy()
    self.RevLayers.reverse()

    self.Loss = MSE()

  def forward(self, x):
    for layer in self.Layers:
      x = layer.forward(x)
    return x

  def loss(self, x, t):
    y = self.forward(x)
    L = self.Loss.forward(y,t)
    return L

  def backward(self):
    dL = self.Loss.backward()
    for layer in self.RevLayers:
      dL = layer.backward(dL)
    return dL

  def update(self, lr=0.01):
    for layer in self.RevLayers:
      layer.update(lr)

  def fit(self, x, t, batch_size=10, epoch=100, lr=0.01):
    b = batch_size
    N = x.shape[0]//b
    Mask = np.arange(x.shape[0]) 
    for e in range(epoch):
      np.random.shuffle(Mask)
      for i in range(N):
        xd = x[Mask[b*i:b*(i+1)]]
        td = t[Mask[b*i:b*(i+1)]]
        L = self.loss(xd,td)
        dL = self.backward()
        self.update(lr=lr)
      print(f"\r loss value = {L}", end="")

In [ ]:
# 학습 데이터 생성

np.random.seed(0) 

X = np.random.randint(100, size=(2000, 6)) / 100


Y_max = np.max(X, axis=1).reshape(-1, 1)  
Y_min = np.min(X, axis=1).reshape(-1, 1) 

Y = np.hstack([Y_max, Y_min])            

In [ ]:
# 모델 학습

model = ANN()

model.fit(X,Y,batch_size = 20, epoch = 1000, lr = 0.001)

 loss value = 0.010597441892958226

In [ ]:
# 모델 테스트

X_test = np.array([[0.3, 0.5, 0.31, 0.55, 0.01, 0.9]])
Y_test = model.forward(X_test)
print(Y_test)

[[0.80265567 0.10084232]]
